<a href="https://colab.research.google.com/github/donbcolab/AIE3/blob/main/Copy_of_paligemma_cnmc_finetune_v6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paligemma Fine Tuning using CNMC dataset

### Setting Up

In [1]:
base_model_name = "google/paligemma-3b-pt-224"
adapter_version = "paligemma-cnmc-ft"
adapter_model_name = f"dwb2023/{adapter_version}"

In [2]:
!pip install -qU bitsandbytes datasets accelerate loralib peft transformers trl

In [3]:
import io
import os
from PIL import Image
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import(
    PaliGemmaForConditionalGeneration, AutoProcessor, TrainingArguments,
    Trainer, BitsAndBytesConfig, BitsAndBytesConfig
)
from datasets import load_dataset, DatasetDict

In [4]:
# verify gpu is available
torch.cuda.is_available()

True

In [5]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

In [6]:
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

## Load Dataset

In [7]:
# Load CNMC Dataset
ds = load_dataset("dwb2023/cnmc-leukemia-2019", split="train")

In [8]:
# Load CNMC Dataset
ds = load_dataset("dwb2023/cnmc-leukemia-2019", split="train")

# Filter records to only include those from fold 0
ds_fold_0 = ds.filter(lambda example: example['fold'] == 0)

# Define the percentage you want to retrieve (e.g., 10%)
percentage = 0.10

# Use train_test_split to get the subset
cnmc_ds = ds_fold_0.train_test_split(test_size=percentage)["test"]

# Columns to remove
cols_remove = ["subject_id", "image_number", "cell_count", "class_label", "fold", "original_image_name", "relative_file_path"]
cnmc_ds = cnmc_ds.remove_columns(cols_remove)

In [9]:
# create train test split with test_size=0.2
train_ds = cnmc_ds.train_test_split(test_size=0.2)

# create test val split
test_val_ds = train_ds["test"].train_test_split(test_size=0.5)

cnmc_ds_dict = DatasetDict({
    "train" : train_ds["train"],
    "test" : test_val_ds["test"],
    "validation" : test_val_ds["train"]
})

cnmc_ds_dict

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 282
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 36
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 35
    })
})

## Collate Data

In [10]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained(base_model_name)

In [11]:
max_seq_length = 128
output_dir = adapter_version

In [12]:
import torch
device = "cuda"

def collate_fn(examples):
  texts = ["Are these cells healthy or cancerous?" for _ in range(len(examples))]
  labels = [example['label'] for example in examples]
  images = [example["image"].convert("RGB") for example in examples]
  # images = [Image.open(io.BytesIO(example['image']['bytes'])).convert("RGB") for example in examples]

  tokens = processor(text=texts, images=images, suffix=labels,
                      return_tensors="pt", padding="longest")
  tokens = tokens.to(torch.bfloat16).to(device)
  return tokens

## Load and Quatize the base Model (bitsandbytes)

In [13]:
from transformers import BitsAndBytesConfig
from peft import get_peft_model, LoraConfig

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_type=torch.bfloat16
)

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)
model = PaliGemmaForConditionalGeneration.from_pretrained(base_model_name, quantization_config=bnb_config, device_map={"":0})
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
#trainable params: 11,298,816 || all params: 2,934,634,224 || trainable%: 0.38501616002417344


Unused kwargs: ['bnb_4bit_compute_type']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 11,298,816 || all params: 2,934,765,296 || trainable%: 0.3850


## Train the Adapter Model (trl)

In [14]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PaliGemmaForConditionalGeneration(
      (vision_tower): SiglipVisionModel(
        (vision_model): SiglipVisionTransformer(
          (embeddings): SiglipVisionEmbeddings(
            (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
            (position_embedding): Embedding(256, 1152)
          )
          (encoder): SiglipEncoder(
            (layers): ModuleList(
              (0-26): 27 x SiglipEncoderLayer(
                (self_attn): SiglipAttention(
                  (k_proj): lora.Linear4bit(
                    (base_layer): Linear4bit(in_features=1152, out_features=1152, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Identity()
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=1152, out_features=8, bias=False)
                    )
                    (lora_B): Mo

In [15]:
model.config

PaliGemmaConfig {
  "_name_or_path": "google/paligemma-3b-pt-224",
  "architectures": [
    "PaliGemmaForConditionalGeneration"
  ],
  "bos_token_id": 2,
  "eos_token_id": 1,
  "hidden_size": 2048,
  "ignore_index": -100,
  "image_token_index": 257152,
  "model_type": "paligemma",
  "pad_token_id": 0,
  "projection_dim": 2048,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  },
  "text_config": {
    "hidden_size": 2048,
    "intermediate_size": 16384,
    "model_type": "gemma",
    "num_attention_heads": 8,
    "num_hidden_layers": 18,
    "num_image_tokens": 256,

In [16]:
from trl import SFTConfig

args=SFTConfig(
    output_dir = adapter_version,
    max_steps=50,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    logging_steps=5,
    eval_strategy="steps",
    eval_steps=5,
    learning_rate=2e-5,
    max_seq_length=max_seq_length,
    bf16=True,
    optim="adamw_hf",
    report_to=["tensorboard"],
    dataset_kwargs={
        "num_of_sequences": 1,
        "chars_per_token": 3.6
    },
    dataset_text_field=None,
    packing=True
)

In [17]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=cnmc_ds_dict['train'],
    eval_dataset=cnmc_ds_dict['validation'],
    data_collator=collate_fn
)

TypeError: SFTTrainer._prepare_dataset() got multiple values for argument 'num_of_sequences'